In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
import os
from os import listdir, makedirs
from os.path import join, isfile, isdir, exists
from datetime import datetime
import pandas as pd
import numpy as np
import math
from tqdm import tqdm

In [4]:
# Input
root_folder = '/content/drive/My Drive/OA_Gihan/OA_all/OA activity data_La Trobe'
ActiGraph_60s_epoch_proc_folder = '/content/drive/My Drive/OA_Gihan/OA_all/OA activity data_La Trobe/Analytics_Scripts/Rashmika_Analysis/processed_data/actigraph/epoch_data/60s_ref'
ActiGraph_60hz_raw_folder = '/content/drive/My Drive/OA_Gihan/OA_all/OA activity data_La Trobe/ActiGraph_raw'

# sleep time input_data
sleep_time_file = '/content/drive/My Drive/OA_Gihan/OA_all/OA activity data_La Trobe/Analytics_Scripts/Rashmika_Analysis/metadata/Copy of record_availability_final.csv'

# Output
Actigraph_output_folder = '/content/drive/My Drive/OA activity data_La Trobe/Analytics_Scripts/Gihan_analysis/processed data/caron_and_montoye'

### Load Sleep Data

In [ ]:
raw_files = [f for f in listdir(ActiGraph_60hz_raw_folder) if isfile(join(ActiGraph_60hz_raw_folder, f))]
epoch_files = [f for f in listdir(ActiGraph_60s_epoch_proc_folder) if isfile(join(ActiGraph_60s_epoch_proc_folder, f))]

data_dict = {f.split()[0]: {'raw': join(ActiGraph_60hz_raw_folder, f)} for f in raw_files}
for ef in epoch_files:
  key = ef.split()[0]
  if key in data_dict.keys():
    data_dict[key]['epoch'] = join(ActiGraph_60s_epoch_proc_folder, ef)

In [ ]:
print(data_dict['OA46'])

{'raw': '/content/drive/My Drive/OA_Gihan/OA_all/OA activity data_La Trobe/ActiGraph_raw/OA46 fixedRAW.csv', 'epoch': '/content/drive/My Drive/OA_Gihan/OA_all/OA activity data_La Trobe/Analytics_Scripts/Rashmika_Analysis/processed_data/actigraph/epoch_data/60s_ref/OA46 fixed60sec_60sREF.csv'}


In [ ]:
sleep_df = pd.read_csv(sleep_time_file)
sleep_df.head(2)

,ID,Participant,activpal_Start_time,activpal_End_time,activpal_Time_difference,actigraph_Start_time,actigraph_End_time,actigraph_Time_difference,Overlap,age_yr,sex,BMI,bodymass_kg,AG_data,AP_data,"Side_TKA (1=L,2=R)",slnumnew,nonwearmins,sleepmins,nonwearpct,justnonwear,justsleep,begin_time,end_time
0,0,HOA1,2019-05-06 14:59:46,2019-05-13 08:48:55,161h 49m 9d,2019-05-06 15:00:00,2019-05-13 07:59:59,160h 59m 59d,160h 59m 59d,52.0,female,76.86261,88.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2019-05-06 15:00:00,2019-05-06 22:00:00
1,0,HOA1,2019-05-06 14:59:46,2019-05-13 08:48:55,161h 49m 9d,2019-05-06 15:00:00,2019-05-13 07:59:59,160h 59m 59d,160h 59m 59d,52.0,female,76.86261,88.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2019-05-07 06:00:00,2019-05-07 22:00:00


### Generate statistical features

In [ ]:
FREQUENCY = 60 #HZ amount of the accelerometer
REFERENCE_EPOCH = 60 #Epoch value that EE is given
TIME_EPOCH_DICT = {
    # 'Epoch1': FREQUENCY * 1,
    # 'Epoch5': FREQUENCY * 5,
    'Epoch30': FREQUENCY * 30,
    # 'Epoch60': FREQUENCY * 60
}

In [ ]:
def caron_hip_ee_prediction_1(row):
  if (epoch_df_current['sex'][row.name])=='female':
    sex = 0
    EE_caron = -0.763 + (0.0491*epoch_df_current['mvm'][row.name]) + (0.063*epoch_df_current['BMI'][row.name]) + (0.141*1) + (0.47*sex)
  elif (epoch_df_current['sex'][row.name])=='male':
    sex = 0
    EE_caron = -0.763 + (0.0491*epoch_df_current['mvm'][row.name]) + (0.063*epoch_df_current['BMI'][row.name]) + (0.141*1) + (0.47*sex)
  return EE_caron


def caron_hip_ee_prediction_2(row):
  if (epoch_df_current['sex'][row.name])=='female':
    sex = 0
    EE_caron = -0.763 + (0.0491*epoch_df_current['mvm'][row.name]) + (0.063*epoch_df_current['BMI'][row.name])  + (0.47*sex)
  elif (epoch_df_current['sex'][row.name])=='male':
    sex = 0
    EE_caron = -0.763 + (0.0491*epoch_df_current['mvm'][row.name]) + (0.063*epoch_df_current['BMI'][row.name]) + (0.47*sex)
  return EE_caron


In [ ]:
for _, row in tqdm(sleep_df.iterrows(), total=sleep_df.shape[0]):
  user_id = row['Participant']
  print(_,user_id)
  if(_<227):continue
  try:
    
    begin_time = row['begin_time']
    end_time = row['end_time']

    age = row['age_yr']
    sex = row['sex']
    BMI = row['BMI']
    mass = row['bodymass_kg']

    height = (mass/BMI)**0.5

    # if('HOA7' != user_id):
    #   continue
    print(user_id)

    # get respective epoch dataset
    epoch_df = pd.read_csv(data_dict[user_id]['epoch'])
    epoch_df = epoch_df[(begin_time < epoch_df['Time_stamp']) & (epoch_df['Time_stamp'] < end_time)]
    # print(list(epoch_df.index)[0])
    # continue
    # get respective raw dataset
    raw_begin_index = list(epoch_df.index)[0] * REFERENCE_EPOCH * FREQUENCY
    raw_data_length = epoch_df.shape[0] * REFERENCE_EPOCH * FREQUENCY
    raw_df_orig = pd.read_csv(data_dict[user_id]['raw'], skiprows=11+raw_begin_index, nrows=raw_data_length, header=None)
    raw_df_orig.columns = ['Timestamp', 'X', 'Y', 'Z']
    # print(raw_df_orig.head(20))

    """
    Process RAW data as required by the model features
    """

    # VM and MANGLE needs for calculations by Staudenmayer model
    # Calculate the vector magnitude from X, Y, Z raw readings
    # Calculate the angle of arcsin from X and VM, arcsin(axis used/vector magnitude)/(pi/2)
    raw_df_orig['vm'] = np.sqrt([(raw_df_orig.X ** 2) + (raw_df_orig.Y ** 2) + (raw_df_orig.Z ** 2)])[0]
    # raw_df_orig['angle'] = (90 * np.arcsin(raw_df_orig.X / raw_df_orig['vm'])) / (math.pi / 2)
    # print(raw_df_orig['vm'].head(20))
    # Calculate ENMO (Euclidean Norm Minus One) reguired for Hilderbrand model
    # raw_df_orig['enmo'] = np.sqrt([(raw_df_orig.X ** 2) + (raw_df_orig.Y ** 2) + (raw_df_orig.Z ** 2)])[0] - 1
    # raw_df_orig.loc[raw_df_orig.enmo < 0, 'enmo'] = 0
    # print(raw_df_orig['enmo'].head(20))


    orientation_roll = np.arctan2(raw_df_orig.Y, raw_df_orig.Z) * 180/math.pi
    orientation_pitch = np.arctan2(raw_df_orig.X,np.sqrt((raw_df_orig.Z*raw_df_orig.Z)+(raw_df_orig.Y*raw_df_orig.Y)))*180/math.pi
    angle = (90 * np.arcsin(raw_df_orig.X / raw_df_orig['vm'])) / (math.pi / 2)

    raw_df_orig['orientation_roll'] = orientation_roll
    raw_df_orig['orientation_pitch'] = orientation_pitch
    raw_df_orig['angle'] = angle


    
    # For each epoch given
    for epoch_name, epoch_duration in TIME_EPOCH_DICT.items():
      print(epoch_name,epoch_duration)
      
      # continue
      # Create output path for the combined file
      output_file_name = join(Actigraph_output_folder, epoch_name,'{}_{}_to_{}_{}.csv'.format(user_id, begin_time, end_time, epoch_name).replace(':', '-').replace(' ', '_'))
      if not exists(join(Actigraph_output_folder, epoch_name)):
        makedirs(join(Actigraph_output_folder, epoch_name))
      
      # Only run for non processed files
      # if exists(output_file_name):
      #   continue

      # Copy raw data
      raw_df = raw_df_orig.copy()
      # print(raw_df.head(10))
      # print(raw_df.index)
      raw_df.reset_index(drop=True, inplace=True)
      # print(raw_df.head(10))
      # print(raw_df.index)
      

      # X shifted needs to calculate covariance for Montoye 2017 model
      raw_df['X_shifted'] = raw_df_orig['X'].shift(epoch_duration)
      raw_df['Y_shifted'] = raw_df_orig['Y'].shift(epoch_duration)
      raw_df['Z_shifted'] = raw_df_orig['Z'].shift(epoch_duration)

      """
      Calculate the statistical inputs (Features)
      """
      # print(raw_df.index // epoch_duration)
      
      hip_grouped_temp = raw_df.groupby(raw_df.index // epoch_duration)
      # k=0
      # for name,group in hip_grouped_temp:
      #   print(name)
      #   print(group)
      #   k=k+1
      #   if(k==100):break
      # break
      aggregated_hip = pd.DataFrame()

      # General stats
      aggregated_hip['mvm'] = hip_grouped_temp['vm'].mean()

      # # Aggregated values required for Hilderbrand model
      # aggregated_hip['enmo'] = hip_grouped_temp['enmo'].mean()
      # aggregated_hip['enmo'] = aggregated_hip['enmo'].fillna(1)

      # Aggregated values required for Montoye Models
      aggregated_hip['XMean'] = hip_grouped_temp['X'].mean()
      aggregated_hip['YMean'] = hip_grouped_temp['Y'].mean()
      aggregated_hip['ZMean'] = hip_grouped_temp['Z'].mean()


      aggregated_hip['XVar'] = hip_grouped_temp['X'].var()
      aggregated_hip['YVar'] = hip_grouped_temp['Y'].var()
      aggregated_hip['ZVar'] = hip_grouped_temp['Z'].var()

      aggregated_hip['orientation_roll_mean'] = hip_grouped_temp['orientation_roll'].mean()
      aggregated_hip['orientation_roll_var'] = hip_grouped_temp['orientation_roll'].var()

      aggregated_hip['orientation_pitch_mean'] = hip_grouped_temp['orientation_pitch'].mean()
      aggregated_hip['orientation_pitch_var'] = hip_grouped_temp['orientation_pitch'].var()

      aggregated_hip['angle_mean'] = hip_grouped_temp['angle'].mean()
      aggregated_hip['angle_var'] = hip_grouped_temp['angle'].var()
      

      aggregated_hip['X10perc'] = hip_grouped_temp['X'].quantile(.1)
      aggregated_hip['X25perc'] = hip_grouped_temp['X'].quantile(.25)
      aggregated_hip['X50perc'] = hip_grouped_temp['X'].quantile(.5)
      aggregated_hip['X75perc'] = hip_grouped_temp['X'].quantile(.75)
      aggregated_hip['X90perc'] = hip_grouped_temp['X'].quantile(.9)

      aggregated_hip['Y10perc'] = hip_grouped_temp['Y'].quantile(.1)
      aggregated_hip['Y25perc'] = hip_grouped_temp['Y'].quantile(.25)
      aggregated_hip['Y50perc'] = hip_grouped_temp['Y'].quantile(.5)
      aggregated_hip['Y75perc'] = hip_grouped_temp['Y'].quantile(.75)
      aggregated_hip['Y90perc'] = hip_grouped_temp['Y'].quantile(.9)

      aggregated_hip['Z10perc'] = hip_grouped_temp['Z'].quantile(.1)
      aggregated_hip['Z25perc'] = hip_grouped_temp['Z'].quantile(.25)
      aggregated_hip['Z50perc'] = hip_grouped_temp['Z'].quantile(.5)
      aggregated_hip['Z75perc'] = hip_grouped_temp['Z'].quantile(.75)
      aggregated_hip['Z90perc'] = hip_grouped_temp['Z'].quantile(.9)

      aggregated_hip['X_cov'] = hip_grouped_temp.apply(lambda x: x['X'].cov(x['X_shifted']))
      aggregated_hip['Y_cov'] = hip_grouped_temp.apply(lambda x: x['Y'].cov(x['Y_shifted']))
      aggregated_hip['Z_cov'] = hip_grouped_temp.apply(lambda x: x['Z'].cov(x['Z_shifted']))

      aggregated_hip['X_cov'] = aggregated_hip['X_cov'].fillna(0)
      aggregated_hip['Y_cov'] = aggregated_hip['Y_cov'].fillna(0)
      aggregated_hip['Z_cov'] = aggregated_hip['Z_cov'].fillna(0)

      aggregated_hip['X_min'] = hip_grouped_temp['X'].min()
      aggregated_hip['Y_min'] = hip_grouped_temp['Y'].min()
      aggregated_hip['Z_min'] = hip_grouped_temp['Z'].min()
      aggregated_hip['X_max'] = hip_grouped_temp['X'].max()
      aggregated_hip['Y_max'] = hip_grouped_temp['Y'].max()
      aggregated_hip['Z_max'] = hip_grouped_temp['Z'].max()

      """
      Repeat rows of epoch file as required to supplement raw data.
      """
      # Number of rows for each epoch
      epoch_df_current = epoch_df.copy()
      # print(epoch_df_current.head())
      epoch_df_current.reset_index(drop=True, inplace=True)
      # print(epoch_df_current.head())
      
      multiplication_factor = int(REFERENCE_EPOCH / (epoch_duration/FREQUENCY))#ex: 60/(1800/60) for 30 epochs, ultimately 60/30
      # print('mul',[multiplication_factor,REFERENCE_EPOCH,epoch_duration,FREQUENCY])
      # print('before',epoch_df_current.head())
      epoch_df_current = epoch_df_current.reindex(np.repeat(epoch_df_current.index.values, multiplication_factor), method='ffill')
      # print('after',epoch_df_current.head())
      epoch_df_current['waist_ee_final'] = epoch_df_current['waist_ee']/multiplication_factor
      print('EE',epoch_df_current['waist_ee'])

      epoch_df_current.reset_index(drop=True, inplace=True)
      

  
      
      # break




    
      
      # epoch_df_current['enmo'] = aggregated_hip['enmo']


      # Montoye models and caron models
      #1 mean
      epoch_df_current['raw_XMean'] = aggregated_hip['XMean']
      epoch_df_current['raw_YMean'] = aggregated_hip['YMean']
      epoch_df_current['raw_ZMean'] = aggregated_hip['ZMean']

      #2 variance
      epoch_df_current['raw_XVar'] = aggregated_hip['XVar']
      epoch_df_current['raw_YVar'] = aggregated_hip['YVar']
      epoch_df_current['raw_ZVar'] = aggregated_hip['ZVar']

      #3 covariance
      epoch_df_current['raw_X_cov'] = aggregated_hip['X_cov']
      epoch_df_current['raw_Y_cov'] = aggregated_hip['Y_cov']
      epoch_df_current['raw_Z_cov'] = aggregated_hip['Z_cov']

      #4 minimum
      epoch_df_current['raw_Xmin'] = aggregated_hip['X_min']
      epoch_df_current['raw_Ymin'] = aggregated_hip['Y_min']
      epoch_df_current['raw_Zmin'] = aggregated_hip['Z_min']

      #5 maximum
      epoch_df_current['raw_Xmax'] = aggregated_hip['X_max']
      epoch_df_current['raw_Ymax'] = aggregated_hip['Y_max']
      epoch_df_current['raw_Zmax'] = aggregated_hip['Z_max']

      #6.1 orientation_roll
      epoch_df_current['raw_orientation_roll_mean'] = aggregated_hip['orientation_roll_mean']
      #7.1 orientation_roll
      epoch_df_current['raw_orientation_roll_var']  = aggregated_hip['orientation_roll_var']

      #6.2 orientation_pitch
      epoch_df_current['raw_orientation_pitch_mean'] = aggregated_hip['orientation_pitch_mean']
      #7.2 orientation_pitch
      epoch_df_current['raw_orientation_pitch_var']  = aggregated_hip['orientation_pitch_var']

      #6.3 angle
      epoch_df_current['raw_angle_mean'] = aggregated_hip['angle_mean']
      #7.3 angle
      epoch_df_current['raw_angle_var']  = aggregated_hip['angle_var']

      #8-12 percentiles
      epoch_df_current['raw_X10perc'] = aggregated_hip['X10perc']
      epoch_df_current['raw_X25perc'] = aggregated_hip['X25perc']
      epoch_df_current['raw_X50perc'] = aggregated_hip['X50perc']
      epoch_df_current['raw_X75perc'] = aggregated_hip['X75perc']
      epoch_df_current['raw_X90perc'] = aggregated_hip['X90perc']

      epoch_df_current['raw_Y10perc'] = aggregated_hip['Y10perc']
      epoch_df_current['raw_Y25perc'] = aggregated_hip['Y25perc']
      epoch_df_current['raw_Y50perc'] = aggregated_hip['Y50perc']
      epoch_df_current['raw_Y75perc'] = aggregated_hip['Y75perc']
      epoch_df_current['raw_Y90perc'] = aggregated_hip['Y90perc']

      epoch_df_current['raw_Z10perc'] = aggregated_hip['Z10perc']
      epoch_df_current['raw_Z25perc'] = aggregated_hip['Z25perc']
      epoch_df_current['raw_Z50perc'] = aggregated_hip['Z50perc']
      epoch_df_current['raw_Z75perc'] = aggregated_hip['Z75perc']
      epoch_df_current['raw_Z90perc'] = aggregated_hip['Z90perc']

      #vm for carons
      epoch_df_current['mvm'] = aggregated_hip['mvm']

      #user_characteristics
      epoch_df_current['sex'] = sex
      epoch_df_current['height'] = height
      epoch_df_current['age'] = age
      epoch_df_current['BMI'] = BMI
      epoch_df_current['mass'] = mass

      # print('epoch_df', epoch_df.shape)
      # print('epoch_df_current', epoch_df_current.shape, multiplication_factor)
      # print('raw', aggregated_hip.shape)
      # print('ag_hip', aggregated_hip.head())
      # print('after',epoch_df_current.head())

      epoch_df_current['EE_caron_1'] = epoch_df_current.apply(caron_hip_ee_prediction_1,axis=1)
      epoch_df_current['EE_caron_2'] = epoch_df_current.apply(caron_hip_ee_prediction_2,axis=1)

      epoch_df_current.to_csv(output_file_name, index=None)
  except Exception as e:
    f = open('errors.txt','w+')
    f.write(str(user_id)+" "+str(e)+" \n")
    f.close()




  0%|          | 0/1351 [00:00<?, ?it/s]

0 HOA1
1 HOA1
2 HOA1
3 HOA1
4 HOA1
5 HOA1
6 HOA1
7 HOA1
8 HOA10
9 HOA10
10 HOA10
11 HOA10
12 HOA10
13 HOA10
14 HOA10
15 HOA11
16 HOA11
17 HOA11
18 HOA11
19 HOA11
20 HOA11
21 HOA11
22 HOA11
23 HOA12
24 HOA12
25 HOA12
26 HOA12
27 HOA12
28 HOA12
29 HOA12
30 HOA12
31 HOA14
32 HOA14
33 HOA14
34 HOA14
35 HOA14
36 HOA14
37 HOA14
38 HOA14
39 HOA15
40 HOA15
41 HOA15
42 HOA15
43 HOA15
44 HOA15
45 HOA15
46 HOA15
47 HOA16
48 HOA16
49 HOA16
50 HOA16
51 HOA16
52 HOA16
53 HOA16
54 HOA17
55 HOA17
56 HOA17
57 HOA17
58 HOA17
59 HOA17
60 HOA17
61 HOA17
62 HOA19
63 HOA19
64 HOA19
65 HOA19
66 HOA19
67 HOA19
68 HOA19
69 HOA19
70 HOA2
71 HOA2
72 HOA2
73 HOA2
74 HOA2
75 HOA2
76 HOA2
77 HOA2
78 HOA20
79 HOA20
80 HOA20
81 HOA20
82 HOA20
83 HOA20
84 HOA20
85 HOA20
86 HOA24
87 HOA24
88 HOA24
89 HOA24
90 HOA24
91 HOA24
92 HOA24
93 HOA24
94 HOA26
95 HOA26
96 HOA26
97 HOA26
98 HOA26
99 HOA26
100 HOA26
101 HOA26
102 HOA27
103 HOA27
104 HOA27
105 HOA27
106 HOA27
107 HOA27
108 HOA27
109 HOA27
110 HOA29
111 HOA29
112 HO



 17%|█▋        | 228/1351 [00:05<00:25, 44.48it/s]

EE 0      1.336129
0      1.336129
1      1.336129
1      1.336129
2      1.369981
         ...   
296    1.336129
297    1.336129
297    1.336129
298    1.336129
298    1.336129
Name: waist_ee, Length: 598, dtype: float64
228 HOA45
HOA45
Epoch30 1800




 17%|█▋        | 228/1351 [00:15<00:25, 44.48it/s]

 17%|█▋        | 229/1351 [00:21<1:29:49,  4.80s/it]

...



100%|█████████▉| 1349/1351 [5:26:33<00:22, 11.39s/it]

1349 OA9
OA9




100%|█████████▉| 1350/1351 [5:26:37<00:09,  9.18s/it]

1350 OA9
OA9




100%|██████████| 1351/1351 [5:26:44<00:00, 14.51s/it]
